In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
def load_data():
    dirs = os.listdir("c:/Users/patel/Documents/GitHub/wizardy-world-word-generation/books/")
    data = []
    
    for file_name in dirs:
        file_name = "./books/" + file_name
        file = open(file_name, 'r', encoding="utf8")
        data.append(file.read())
        file.close()
    
    return data

In [24]:
def clean_data(data):
    words = []
    for i in data:
        words.append(re.findall(r'\w+', i))
    cleaned_data = []
    for i in words:
        for word in i:
            if word.isalpha():
                cleaned_data.append(word.lower())
    return cleaned_data

In [25]:
def sequence_data(cleaned_data):
    x = len(cleaned_data)//51
    cleaned_data = cleaned_data[:51*x]
    sequences = [cleaned_data[i:i+51] for i in range(0,len(cleaned_data),51)]
    return sequences
        

In [26]:
data = load_data()
cleaned_data = clean_data(data)
sequenced_data = sequence_data(cleaned_data)

In [27]:
len(sequenced_data[0])

51